In [ ]:
pip install gradio

# imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import gradio as gr
from PIL import Image
import os

# Data Preparation and Model Training

In [ ]:
def train_model():
    print("1. Loading and Preprocessing Data...")
    # Load MNIST dataset
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Normalize data to be between 0 and 1
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Reshape data to fit CNN input (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # Convert labels to One-Hot Encoding
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    # Build the CNN model
    print("2. Building the Model...")
    model = keras.Sequential(
        [
            layers.Input(shape=(28, 28, 1)),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    # Compile the model
    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )

    # Train the model
    print("3. Training the Model (Please wait)...")
    batch_size = 128
    epochs = 5  # Number of training epochs
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.1,
        verbose=1
    )

    # Evaluate final accuracy
    score = model.evaluate(x_test, y_test, verbose=0)
    print(f"\nTraining Completed!\nFinal Test Accuracy: {score[1]:.4f}")

    # Save the trained model
    model.save("mnist_model.h5")
    print("Model saved as 'mnist_model.h5'")
    return model


# Load the model if it exists, otherwise train a new one
if os.path.exists("mnist_model.h5"):
    model = keras.models.load_model("mnist_model.h5")
    print("Model loaded from 'mnist_model.h5'")
else:
    model = train_model()

1. Loading and Preprocessing Data...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
2. Building the Model...
3. Training the Model (Please wait)...
Epoch 1/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 38s 87ms/step - accuracy: 0.7638 - loss: 0.7556 - val_accuracy: 0.9800 - val_loss: 0.0813
Epoch 2/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 41s 88ms/step - accuracy: 0.9621 - loss: 0.1254 - val_accuracy: 0.9853 - val_loss: 0.0546
Epoch 3/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 42s 90ms/step - accuracy: 0.9730 - loss: 0.0894 - val_accuracy: 0.9893 - val_loss: 0.0445
Epoch 4/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 40s 88ms/step - accuracy: 0.9774 - loss: 0.0732 - val_accuracy: 0.9905 - val_loss: 0.0391
Epoch 5/5
422/422 ━━━━━━━━━━━━━━━━━━━━ 39s 84ms/step - accuracy: 0.9800 - loss: 0.0621 - val_accuracy: 0.9897 - val_loss: 0.0365



Training Completed!
Final Test Accuracy: 0.9882
Model saved as 'mnist_model.h5'


# Image Prediction Function

In [ ]:
def predict_digit(image):
    # Convert the input array to a grayscale PIL image
    img = Image.fromarray(image).convert('L')

    # Resize the image to 28x28 pixels
    img = img.resize((28, 28))
    img_array = np.array(img)

    # Smart color inversion: if the image has a light background (white) and dark digit
    if np.mean(img_array) > 127:
        print("Detected light background. Inverting colors...")
        img_array = 255 - img_array

    # Prepare the image for the model
    processed_img = img_array.astype("float32") / 255
    processed_img = processed_img.reshape(1, 28, 28, 1)

    # Make prediction
    prediction = model.predict(processed_img)
    predicted_digit = np.argmax(prediction)
    confidence = np.max(prediction) * 100

    # Create a figure for display
    fig, ax = plt.subplots(figsize=(3, 3))
    ax.imshow(img_array, cmap='gray')
    ax.set_title(f"Prediction: {predicted_digit}\nConfidence: {confidence:.2f}%")
    ax.axis('off')
    plt.close(fig)

    return fig, f"Predicted Digit: {predicted_digit} (Confidence: {confidence:.2f}%)"

# creat GUI WITH Gradio

In [ ]:

with gr.Blocks() as demo:
    gr.Markdown("# MNIST Digit Recognizer with CNN")
    gr.Markdown("Upload an image of a digit (preferably 28x28, white background with black digit or vice versa). The model will predict the digit!")

    with gr.Row():
        image_input = gr.Image(label="Upload Image", type="numpy")
        output_plot = gr.Plot(label="Processed Image with Prediction")

    output_text = gr.Textbox(label="Prediction Result")

    predict_btn = gr.Button("Predict Digit")
    predict_btn.click(predict_digit, inputs=image_input, outputs=[output_plot, output_text])

    gr.Markdown("---")
    gr.Markdown("If the model isn't trained, click below to train it (this may take a few minutes).")
    train_btn = gr.Button("Train Model")
    train_btn.click(lambda: train_model(), inputs=[], outputs=[])

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5c17f6ae31120a57c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
